Calculating rare k-mers

In [ ]:
# For its use in colab notebook
!pip install jbrowse-jupyter

In [ ]:
#in Google Colab, it seems that we can't use Flask 2.1.3, so we'll force it to use 1.1.4
!pip install Flask==1.1.4

In [ ]:
# For its use in colab notebook
!pip install pandas
import pandas as pd

In [ ]:
#refseq_name = 'NZ_CP007470.1' #it would be nice to extract this from fasta or gff (which is what we do below when reading the fasta file line by line)

All of the data we are using came originally from this NCBI link: https://www.ncbi.nlm.nih.gov/genome/?term=NZ_CP007470.1. Things I did to make our lives easier:
* Created a bgzipped and faidx indexed (SAMtools) copy of the fasta sequence
* Created a bgzipped and tabix indexed copy of the GFF (which contains the gene annotations)
* Created an unzipped copy of the fasta sequence (to make it easier for us to read)
* Plopped all of this data in a AWS S3 bucket with CORS enabled for public access

Here we are creating an empty JBrowse configuration option, telling it that it will be a linear genome view (LGV) as opposed to a circular genome view (CGV) and setting the assembly, telling where to find the indexed fasta file.

Using the "base_url" variable here assumes that the following files are present:
* `base_url + ".fa"` (the uncompressed fasta file)
* `base_url + ".fa.gz"` (the bgzipped fasta file)
* `base_url + ".fa.gz.fai"` (the faidx index file)
* `base_url + ".fa.gz.gzi"` (the bgzip index file)
* `base_url + ".gff.gz"` (the bgzipped gff file)
* `base_url + ".gff.gz.tbi"` (the tabix index of the gff file)

Using this approach, we can switch to any other bacterial genome by proving all of these files, and only changing the base_url value in this page.

In [ ]:
base_url = 'https://s3.amazonaws.com/wormbase-modencode/test/h_influenzae/h_influenzae_477'

In [ ]:
fasta_url = base_url + '.fa' #unzipped--this is what we are going to read in this script

This is the "magic" for reading the contents from a URL.

In [ ]:
import requests

r = requests.get(fasta_url, stream=True)

if r.encoding is None:
  r.encoding = 'utf-8'

refseq_name = ''
seq = '' # this is a string that will contain the entire chromosome sequence
firstline = True
for line in r.iter_lines(decode_unicode=True):
  if line:
    if firstline:
      space = line.index(" ")
      refseq_name = line[1:space]
      print(refseq_name)
      firstline = False
      continue
    if line.find(">") > -1: #stop after reading the first sequence if there is more than one
      break
    seq = seq + line.rstrip("\n") #strip the carriage returns off the end of each line in the fasta before concatenating

print(len(seq))
    
    

In [ ]:
# don't actally need create here--that's only good for "built in" geneomes hg19, hg38
from jbrowse_jupyter import create, launch

In [ ]:
import jbrowse_jupyter
config = jbrowse_jupyter.JBrowseConfig(view="LGV",)
config.set_assembly(base_url + '.fa.gz', # bgzipped and then samtools faidx
                    name=refseq_name)

Here we are creating our first JBrowse 2 track from the GFF3 file I mentioned above. It's pretty straight forward, just giving it a url for the data and a name that will be displayed in JBrowse 2 (the "name") and a track_id, which is what we use to refer to it programatically below.

In [ ]:
#add gene track
geneGFFUrl = base_url + '.gff.gz'
config.add_track(geneGFFUrl, name="Genes",track_id="Genes")

Now to start looking for k-mers. We loop through every position in the sequence that we got above (`seq`) and get the k-mer by taking the substring starting at the current position in the string (`seq[i:i+k]`). If we've seen that k-mer before, we just add to the count and if not, we start a count at 1.

In [ ]:
k = 6 #start with hexamers
kmer_list = {}
kmer_count = {}

i = 1 #start counting bases at 1 (yeah, I know)
for bp in seq: #loop over every base in the sequence
  kmer_list[i] = seq[i:i+k] 
  #keep track of how many times we've seen a give n-mer
  if (kmer_list[i] in kmer_count):
    kmer_count[kmer_list[i]] += 1
  else:
    kmer_count[kmer_list[i]] = 1
  i += 1
  if(i>len(seq)-k):  # when this is true, we've seen the last n-mer
    #print(i)
    print(kmer_count)
    break
    
#print(nmer_count['CGTCCG'])

Now make dictionary out of our results, giving it an index, and then create a DataFrame out of it.

In [ ]:
index = 0
kmer = {}
count = {}
for km in kmer_count:
  kmer[index]=km
  count[index]=kmer_count[km]
  index += 1

count_data = {'kmer':kmer,
              'count':count}
df = pd.DataFrame(count_data)
print(df)

Sort the DataFrame (which goes into a new DataFrame)

In [ ]:
#now sort them and find the least common
sorted_kmers = df.sort_values(by=['count'])
print(sorted_kmers)

Look at statistics just for being interesting.

In [ ]:
print('average')
print(df['count'].mean())
print('stdev')
print(df['count'].std())

Create a function that will create a DataFrame that JBrowse can use when given the rank of the "rare-ness" in the sorted DataFrame (ie, 0 is the most rare, 1 is the next most rare, etc).

In [ ]:
def nth_least_common(rank):
#make features out of the least common nmer
#find them in the original list, then use the index to create coordinates
  least_common = sorted_kmers.iloc[rank].kmer

  refName = []
  start = []
  end = []
  name = []
  for i in range(0,len(seq)):
    if least_common == seq[i:i+k]:
      print(i)
      print(kmer_list[i])
      refName.append(refseq_name)
      start.append(i)
      end.append(i+k)
      name.append(least_common + ':' + str(i))
  least_common_track_data = {'refName' : refName,
                           'start' : start,
                           'end' : end,
                           'name' : name}
  lc_df = pd.DataFrame(least_common_track_data)
  return lc_df

Use that function to create a few track DataFrames.

In [ ]:
zeroth_least_common_df = nth_least_common(0)
print(zeroth_least_common_df)

In [ ]:
first_least_common_df = nth_least_common(1)
print(first_least_common_df)

Finally, add those DataFrame tracks, set the initial location to the first most rare k-mer and open JBrowse with the gene track from above and the k-mer tracks open.

In [ ]:
config.add_df_track(zeroth_least_common_df, 'least common k mer',track_id='least_common',overwrite=True)
config.add_df_track(first_least_common_df,'next least common k mer',track_id='next_least_common',overwrite=True)
location = refseq_name + ":" + str(zeroth_least_common_df.loc[0,'start']-1000) + '..' + str(zeroth_least_common_df.loc[0,'start']+1000) #set the initial location centered on the first feature
config.set_location(location)
config.set_default_session(['Genes','df_gc_skew','least_common','next_least_common'], False)
full_conf = config.get_config()
launch(full_conf, port=3003, height=600)